<a href="https://colab.research.google.com/github/RubaHesham11/paper_topic_classification/blob/main/paper_topic_classification_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rubahesham","key":"6bd74e8c8e152cc977c8b1d3804b080a"}\r\n'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d shivanandmn/multilabel-classification-dataset

  0% 0.00/11.4M [00:00<?, ?B/s]
100% 11.4M/11.4M [00:00<00:00, 281MB/s]


In [ ]:
!unzip multilabel-classification-dataset.zip

Archive:  multilabel-classification-dataset.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

class LinearNN:
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size)


    def train(self, X, T, epochs, lr):
        for epoch in range(epochs):
            # Forward propagation
            y_pred = self.predict(X)

            # Compute gradient
            d_weights = np.dot(X.T, (y_pred - T) * sigmoid_derivative(y_pred))


            # Update weights and bias
            self.weights -= lr * d_weights


            if epoch % 100 == 0:
                loss = np.mean(-T * np.log(y_pred) - (1 - T) * np.log(1 - y_pred))
                print(f'Loss at epoch {epoch}: {loss}')

    def predict(self, X):
        return sigmoid(np.dot(X, self.weights) )




In [ ]:
# Load the dataset
data = pd.read_csv('/content/train.csv')



In [ ]:
data

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [ ]:
print(data.columns)

Index(['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
      dtype='object')


In [ ]:
# Prepare the features
X = data.drop(['ID','Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance'], axis=1)

y = data.drop(['ID', 'TITLE', 'ABSTRACT'], axis=1)


In [ ]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
print(type(X_train))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
print(X_train.dtypes)

TITLE       object
ABSTRACT    object
dtype: object


In [ ]:
# Assuming 'TITLE' and 'ABSTRACT' are text columns
tfidf = TfidfVectorizer(max_features=5000)  # Example max_features
title_abstract_features_train = tfidf.fit_transform(X_train[['TITLE', 'ABSTRACT']].apply(lambda x: ' '.join(x), axis=1)).toarray()

# Convert other columns to a NumPy array
numeric_features_train = X_train.drop(['TITLE', 'ABSTRACT'], axis=1).to_numpy()

# Combine numerical and text features
X_train_processed = np.hstack((title_abstract_features_train, numeric_features_train))

print("Processed X_train shape:", X_train_processed.shape)



Processed X_train shape: (16777, 5000)


In [ ]:
X_train_processed_with_bias = np.hstack((np.ones((X_train_processed.shape[0], 1)), X_train_processed))

print("Processed X_train with bias shape:", X_train_processed_with_bias.shape)

Processed X_train with bias shape: (16777, 5001)


In [ ]:
print("y_train:", y_train.shape)

y_train: (16777, 6)


In [ ]:
np.random.seed(0)
model = LinearNN(X_train_processed.shape[1], y_train.shape[1])
model.train(X_train_processed, y_train, epochs=100, lr=0.001)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Loss at epoch 0: Computer Science        2.065603
Physics                 2.579317
Mathematics             2.716272
Statistics              2.666964
Quantitative Biology    3.457715
Quantitative Finance    3.477133
dtype: float64


In [ ]:
print("Model weights:")
print(model.weights)

print("Predictions on training data:")
print(np.round(model.predict(X_train_processed)))



Model weights:
[[ 0.52307325  0.6743092   0.51655925  0.50158896  0.35922702  0.57473401]
 [ 0.39207783  0.91560642  0.91911055  0.34168655  0.75856442  0.49659392]
 [ 0.53643648  0.93390946  0.04056943  0.05713652 -0.00182638  0.80829471]
 ...
 [ 0.27907859  0.33107822  0.51447273  0.67500614  0.77462837  0.12526933]
 [ 0.34964082  0.48484934  0.44804145  0.65816285  0.79635886  0.44980312]
 [ 0.74634249  0.66170959  0.45713037  0.78111504  0.68025332  0.58484245]]
Predictions on training data:
[[0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [ ]:
# Evaluate the model on the test data
title_abstract_features_test = tfidf.transform(X_test[['TITLE', 'ABSTRACT']].apply(lambda x: ' '.join(x), axis=1)).toarray()
numeric_features_test = X_test.drop(['TITLE', 'ABSTRACT'], axis=1).to_numpy()
X_test_processed = np.hstack((title_abstract_features_test, numeric_features_test))


In [ ]:
# Predict labels for the test data
predictions = np.round(model.predict(X_test_processed))


In [ ]:
# Calculate accuracy
accuracy = np.mean((predictions == np.array(y_test)))
print("Test Accuracy with threshold", 0.5, ":", accuracy)

Test Accuracy with threshold 0.5 : 0.8092570520460866
